In [1]:
using LinearAlgebraicRepresentation
Lar=LinearAlgebraicRepresentation
function merge_vertices(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)
	# remove vertices congruent to a single representative
    todelete = []
    i = 1
    for vi in 1:vertsnum
        if !(vi in todelete)
            
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            newverts[nearvs] .= i
            nearvs = setdiff(nearvs, vi)
            todelete = union(todelete, nearvs)
            @show todelete
            i = i + 1
            
        end
    end
    nV = V[setdiff(collect(1:vertsnum), todelete), :]

    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))

    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    for ei in 1:nedgenum
    	begin
        	nEV[ei, collect(nedges[ei])] .= 1
        	nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    for e in 1:nedgenum
    	v1,v2 = findnz(nEV[e,:])[1]
    	nEV[e,v1] = -1; nEV[e,v2] = 1
    end

    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]


    visited = []
    function filter_fn(face)

        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)

        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end

    nfaces = filter(filter_fn, faces)

    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))

    for fi in 1:nfacenum
        for edge in nfaces[fi]
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end

    return Lar.Points(nV), nEV, nFE
end

merge_vertices (generic function with 2 methods)

In [2]:
using SparseArrays
using LinearAlgebraicRepresentation
using LinearAlgebra
using NearestNeighbors
using BenchmarkTools

Lar=LinearAlgebraicRepresentation



V = Float64[
        0 0 0; 0 1 0;
        1 1 0; 1 0 0;
        0 0 1; 0 1 1;
        1 1 1; 1 0 1
    ]

    EV = sparse(Int8[
        -1  1  0  0  0  0  0  0;
        0 -1  1  0  0  0  0  0;
        0  0 -1  1  0  0  0  0;
        -1  0  0  1  0  0  0  0;
        -1  0  0  0  1  0  0  0;
        0 -1  0  0  0  1  0  0;
        0  0 -1  0  0  0  1  0;
        0  0  0 -1  0  0  0  1;
        0  0  0  0 -1  1  0  0;
        0  0  0  0  0 -1  1  0;
        0  0  0  0  0  0 -1  1;
        0  0  0  0 -1  0  0  1;
    ])

    FE = sparse(Int8[
        1  1  1 -1  0  0  0  0  0  0  0  0;
        0  0  0  0  0  0  0  0 -1 -1 -1  1;
        -1  0  0  0  1 -1  0  0  1  0  0  0;
        0 -1  0  0  0  1 -1  0  0  1  0  0;
        0  0 -1  0  0  0  1 -1  0  0  1  0;
        0  0  0  1 -1  0  0  1  0  0  0 -1;
    ])
@time merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-4)

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
  1.911793 seconds (4.71 M allocations: 241.526 MiB, 5.11% gc time)


([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  -1
  [1,  2]  =  1
  [4,  2]  =  -1
  [1,  3]  =  1
  [5,  3]  =  -1
  [1,  4]  =  -1
  [6,  4]  =  1
  [3,  5]  =  1
  [6,  5]  =  -1
  [3,  6]  =  -1
  [4,  6]  =  1
  [4,  7]  =  -1
  [5,  7]  =  1
  [5,  8]  =  -1
  [6,  8]  =  1
  [2,  9]  =  -1
  [3,  9]  =  1
  [2, 10]  =  -1
  [4, 10]  =  1
  [2, 11]  =  -1
  [5, 11]  =  1
  [2, 12]  =  1
  [6, 12]  =  -1)

In [3]:
@btime merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-4)

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

...

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any

Excessive output truncated after 524291 bytes.

Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete = Any[]
todelete

([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [1 , 1]  =  -1
  [4 , 1]  =  -1
  [5 , 1]  =  -1
  [1 , 2]  =  1
  [2 , 2]  =  -1
  [6 , 2]  =  -1
  [2 , 3]  =  1
  [3 , 3]  =  -1
  [7 , 3]  =  -1
  [3 , 4]  =  1
  [4 , 4]  =  1
  [8 , 4]  =  -1
  [5 , 5]  =  1
  [9 , 5]  =  -1
  [12, 5]  =  -1
  [6 , 6]  =  1
  [9 , 6]  =  1
  [10, 6]  =  -1
  [7 , 7]  =  1
  [10, 7]  =  1
  [11, 7]  =  -1
  [8 , 8]  =  1
  [11, 8]  =  1
  [12, 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  -1
  [1,  2]  =  1
  [4,  2]  =  -1
  [1,  3]  =  1
  [5,  3]  =  -1
  [1,  4]  =  -1
  [6,  4]  =  1
  [3,  5]  =  1
  [6,  5]  =  -1
  [3,  6]  =  -1
  [4,  6]  =  1
  [4,  7]  =  -1
  [5,  7]  =  1
  [5,  8]  =  -1
  [6,  8]  =  1
  [2,  9]  =  -1
  [3,  9]  =  1
  [2, 10]  =  -1
  [4, 10]  =  1
  [2, 11]  =  -1
  [5, 11]  =  1
  [2, 12]  =  1
  [6, 12]  =  -1)

In [7]:
using SharedArrays
using Distributed
addprocs(1)
@everywhere using LinearAlgebraicRepresentation
@everywhere Lar=LinearAlgebraicRepresentation

function merge_vertices(V::Lar.Points, EV::Lar.ChainOp, FE::Lar.ChainOp, err=1e-4)
    vertsnum = size(V, 1)
    #@show vertsnum
    edgenum = size(EV, 1)
    facenum = size(FE, 1)
    newverts = zeros(Int, vertsnum)
    newvertsshared= convert(SharedArray,newverts)
    i=[1]
    ishared= convert(SharedArray,i)
    # KDTree constructor needs an explicit array of Float64
    V = Array{Float64,2}(V)
    W = convert(Lar.Points, LinearAlgebra.transpose(V))
    kdtree = KDTree(W)
	# remove vertices congruent to a single representative
    #todelete = SharedArray{Int}
    todelete=[0]
    todeleteshared= convert(SharedArray,todelete)
    flag=[1]
    flagshared= convert(SharedArray,flag)
    
    
        @sync @distributed for vi in 1:vertsnum
        if !(vi in todeleteshared)
            
            nearvs = Lar.inrange(kdtree, V[vi, :], err)
            #newverts[nearvs] .= i
            while(flagshared[1]==0)
                end
            if(flagshared[1]==1)
                flagshared[1]=0
                newvertsshared[nearvs] .= ishared[1]
                ishared[1]+=1
            end
            flagshared[1]=1
            nearvs = setdiff(nearvs,vi)
            todeleteshared =  union(todeleteshared, nearvs)
            
            @show vi
            @show ishared[1]
            @show newvertsshared
            
            #@show todelete,newvertsshared,vi
        end
       
        
    end
    newverts= convert(Array,newvertsshared)
    todelete= convert(SharedArray,todeleteshared)
    @show newvertsshared
    nV = V[setdiff(collect(1:vertsnum), todelete), :]

    # translate edges to take congruence into account
    edges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    oedges = Array{Tuple{Int, Int}, 1}(undef, edgenum)
    for ei in 1:edgenum
        v1, v2 = EV[ei, :].nzind
        edges[ei] = Tuple{Int, Int}(sort([newverts[v1], newverts[v2]]))
        oedges[ei] = Tuple{Int, Int}(sort([v1, v2]))
    end
    nedges = union(edges)
    # remove edges of zero length
    nedges = filter(t->t[1]!=t[2], nedges)
    nedgenum = length(nedges)
    nEV = spzeros(Int8, nedgenum, size(nV, 1))

    etuple2idx = Dict{Tuple{Int, Int}, Int}()
    for ei in 1:nedgenum
    	begin
        	nEV[ei, collect(nedges[ei])] .= 1
        	nEV
        end
        etuple2idx[nedges[ei]] = ei
    end
    for e in 1:nedgenum
    	v1,v2 = findnz(nEV[e,:])[1]
    	nEV[e,v1] = -1; nEV[e,v2] = 1
    end

    # compute new faces to take congruence into account
    faces = [[
        map(x->newverts[x], FE[fi, ei] > 0 ? oedges[ei] : reverse(oedges[ei]))
        for ei in FE[fi, :].nzind
    ] for fi in 1:facenum]


    
   

    nfaces = filter(filter_fn, faces)

    nfacenum = length(nfaces)
    nFE = spzeros(Int8, nfacenum, size(nEV, 1))

     for fi in 1:nfacenum
         for edge in nfaces[fi]
            #@show edge,fi
            ei = etuple2idx[Tuple{Int, Int}(sort(collect(edge)))]
            nFE[fi, ei] = sign(edge[2] - edge[1])
        end
    end

    return Lar.Points(nV), nEV, nFE
end

merge_vertices (generic function with 2 methods)

In [8]:
 function filter_fn(face)
        visited = []
        verts = []
        map(e->verts = union(verts, collect(e)), face)
        verts = Set(verts)

        if !(verts in visited)
            push!(visited, verts)
            return true
        end
        return false
    end

filter_fn (generic function with 1 method)

In [9]:
@time merge_vertices(Lar.Points(V),Lar.ChainOp(EV),Lar.ChainOp(FE),1e-4)

      From worker 2:	vi = 5
      From worker 2:	ishared[1] = 2
      From worker 2:	newvertsshared = [0, 0, 0, 0, 1, 0, 0, 0]
      From worker 2:	vi = 6
      From worker 2:	ishared[1] = 3
      From worker 2:	newvertsshared = [0, 0, 0, 0, 1, 2, 0, 0]
      From worker 2:	vi = 7
      From worker 2:	ishared[1] = 4
      From worker 2:	newvertsshared = [0, 0, 0, 0, 1, 2, 3, 0]
      From worker 2:	vi = 8
      From worker 2:	ishared[1] = 5
      From worker 2:	newvertsshared = [0, 0, 0, 0, 1, 2, 3, 4]
      From worker 3:	vi = 1
      From worker 3:	ishared[1] = 6
      From worker 3:	newvertsshared = [5, 0, 0, 0, 1, 2, 3, 4]
      From worker 3:	vi = 2
      From worker 3:	ishared[1] = 7
      From worker 3:	newvertsshared = [5, 6, 0, 0, 1, 2, 3, 4]
      From worker 3:	vi = 3
      From worker 3:	ishared[1] = 8
      From worker 3:	newvertsshared = [5, 6, 7, 0, 1, 2, 3, 4]
      From worker 3:	vi = 4
      From worker 3:	ishared[1] = 9
      From worker 3:	newvertsshared = [5, 6, 7,

([0.0 0.0 0.0; 0.0 1.0 0.0; … ; 1.0 1.0 1.0; 1.0 0.0 1.0], 
  [5 , 1]  =  -1
  [9 , 1]  =  -1
  [12, 1]  =  -1
  [6 , 2]  =  -1
  [9 , 2]  =  1
  [10, 2]  =  -1
  [7 , 3]  =  -1
  [10, 3]  =  1
  [11, 3]  =  -1
  [8 , 4]  =  -1
  [11, 4]  =  1
  [12, 4]  =  1
  [1 , 5]  =  -1
  [4 , 5]  =  -1
  [5 , 5]  =  1
  [1 , 6]  =  1
  [2 , 6]  =  -1
  [6 , 6]  =  1
  [2 , 7]  =  1
  [3 , 7]  =  -1
  [7 , 7]  =  1
  [3 , 8]  =  1
  [4 , 8]  =  1
  [8 , 8]  =  1, 
  [1,  1]  =  1
  [3,  1]  =  -1
  [1,  2]  =  1
  [4,  2]  =  -1
  [1,  3]  =  1
  [5,  3]  =  -1
  [1,  4]  =  -1
  [6,  4]  =  1
  [3,  5]  =  -1
  [6,  5]  =  1
  [3,  6]  =  1
  [4,  6]  =  -1
  [4,  7]  =  1
  [5,  7]  =  -1
  [5,  8]  =  1
  [6,  8]  =  -1
  [2,  9]  =  -1
  [3,  9]  =  1
  [2, 10]  =  -1
  [4, 10]  =  1
  [2, 11]  =  -1
  [5, 11]  =  1
  [2, 12]  =  1
  [6, 12]  =  -1)

      From worker 2:	fatal: error thrown and no exception handler available.
      From worker 3:	fatal: error thrown and no exception handler available.
      From worker 3:	jl_mutex_unlock at /cygdrive/d/buildbot/worker/package_win64/build/src\locks.h:143 [inlined]
      From worker 3:	jl_task_get_next at /cygdrive/d/buildbot/worker/package_win64/build/src\partr.c:476
      From worker 2:	jl_mutex_unlock at /cygdrive/d/buildbot/worker/package_win64/build/src\locks.h:143 [inlined]
      From worker 2:	jl_task_get_next at /cygdrive/d/buildbot/worker/package_win64/build/src\partr.c:476
      From worker 2:	poptask at .\task.jl:704
      From worker 2:	wait at .\task.jl:712 [inlined]
      From worker 2:	task_done_hook at .\task.jl:442
      From worker 2:	jl_apply at /cygdrive/d/buildbot/worker/package_win64/build/src\julia.h:1690 [inlined]
      From worker 2:	jl_finish_task at /cygdrive/d/buildbot/worker/package_win64/build/src\task.c:198
      From worker 2:	start_task at /cygdrive/d